# 数据描述统计

根据数据的类型与意义数据可分为：
- 数值数据（Numerical data）；
- 分类数据（Categorical data ）；
- 有序数据（Ordinal data）；
- 文本数据（text data）。

`Series`与`DataFrame`对象用来组织管理数据集，需要掌握数据集中包含何种数据，数据的基本概况如何？本节会介绍一些 Pandas 中基本的数据查看函数或方法。

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## 查看数据特征

这里仍然使用上节的饭店消费示例，但在创建数据集时加入了一些非数字值：

In [27]:
from datetime import datetime
# 时间
times = [datetime(2018, 10, 10, 6), datetime(2018, 10, 10, 7), 
         datetime(2018, 10, 10, 8), datetime(2018, 10, 10, 11), 
         datetime(2018, 10, 10, 12), datetime(2018, 10, 10, 13), 
         datetime(2018, 10, 10, 18), datetime(2018, 10, 10, 19), 
         datetime(2018, 10, 10, 20), datetime(2018, 10, 10, 21)]
# 餐时标识，顾客人数、平均消费、总消费
meals = np.array(['B', 'B', 'B', 'L', 'L', 'L', 'D', 'D', 'D', ''])
customers = np.array([60, 70, 65, 100, 230, 150, 100, 300, np.nan, 300])
costs = np.array([6.5, 7, 8, 24, 23, 26, 45, 55.5, 45, np.nan])
total_costs = customers * costs

# 创建Series、DataFrame对象
ser = pd.Series(customers, index=times)
data = {'meals': meals, 'customers': customers, 'costs': costs, 'total_costs': total_costs}
df = pd.DataFrame(data, index=times)

使用`info()`来查看数据集的数据及数据类型：

In [28]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10 entries, 2018-10-10 06:00:00 to 2018-10-10 21:00:00
Data columns (total 4 columns):
meals          10 non-null object
customers      9 non-null float64
costs          9 non-null float64
total_costs    8 non-null float64
dtypes: float64(3), object(1)
memory usage: 937.0 bytes


根据数据集的数据类型以及意义，可以把`df`对象的数据分为：
- 餐时标识（`meals`）：字符串，分别代表早餐、午餐与晚餐，属于分类数据，
- 顾客人数：整数类型，表示人数，属于数值数据
- 平均消费、总消费：浮点数类型，属于数值数据，且为连续数据
- 日期索引，可认为是特殊的有序数据。

在数据收集过程中，经常会遇到数据缺失的情况。对于数值型数据，常使用`NaN (Not a Number)`来表示缺失值，对于日期型数据`datetime64`则使用`NaT (Not a Time)`来表示。Pandas中，可以使用序列或数据帧的`isnull()`或`isna()`方法来查看：

In [29]:
ser.isnull()

2018-10-10 06:00:00    False
2018-10-10 07:00:00    False
2018-10-10 08:00:00    False
2018-10-10 11:00:00    False
2018-10-10 12:00:00    False
2018-10-10 13:00:00    False
2018-10-10 18:00:00    False
2018-10-10 19:00:00    False
2018-10-10 20:00:00     True
2018-10-10 21:00:00    False
dtype: bool

In [33]:
df.isna()

,meals,customers,costs,total_costs
2018-10-10 06:00:00,False,False,False,False
2018-10-10 07:00:00,False,False,False,False
2018-10-10 08:00:00,False,False,False,False
2018-10-10 11:00:00,False,False,False,False
2018-10-10 12:00:00,False,False,False,False
2018-10-10 13:00:00,False,False,False,False
2018-10-10 18:00:00,False,False,False,False
2018-10-10 19:00:00,False,False,False,False
2018-10-10 20:00:00,False,True,False,True
2018-10-10 21:00:00,False,False,True,True


`Series`与`DataFrame`结构类型中的数据是可重复的，使用`duplicated()`方法来检查是否有重复样本，其使用语法为：
```python
s.duplicated(keep='first')                  # Series对象
df.duplicated(subset=None, keep='first')         # DataFrame对象
```
参数说明
- `subset`，是否检查指定列
- `keep='first'`：`first`将后出现判断为重复值。`last`则反之。

In [ ]:
ser.duplicated()

In [ ]:
df.duplicated(subset=['meals', 'costs'])

使用序列对象的`unique()`方法可以返回唯一值：

In [ ]:
ser.unique()

In [ ]:
df['meals'].unique()

对于分类数据可以使用序列对象的`value_counts()`方法进行频次统计，返回一个序列对象，不同值的频次。其使用语法为：
```python
s.value_counts(normalize=False, sort=True, ascending=False, bins=None, dropna=True)
```
参数说明
- `normalize=False`，如果为真则返回频次占比；
- `sort=True`，进行排序；
- `ascending=False`，排列次序，缺省为降序；
- `bins=None`，仅用于数值型数据，对数据进行分箱计数；
- `dropna=True`，是否包含空值；

In [34]:
# 分类数据
df['meals'].value_counts()

B    3
D    3
L    3
     1
Name: meals, dtype: int64

In [35]:
# 数值数据
df['costs'].value_counts(bins=4)

(43.25, 55.5]                 3
(18.75, 31.0]                 3
(6.449999999999999, 18.75]    3
(31.0, 43.25]                 0
Name: costs, dtype: int64

## 查看数值统计信息

对于数值型数据，Pandas 提供有如下描述性统计方法：
- `count()`，非 NA 值的数量
- `min(), max()`，计算最小值和最大值
- `argmin(), argmax()`，计算最小值和最大值的索引位置（整数）
- `idxmin(), idxmax()`，计算最小值和最大值的索引值
- `quantile()`，计算样本的百分位数（0到1）
- `sum()`，值得总和
- `mean()`，平均值
- `median()`，中位数
- `mad()`，根据平均值计算平均绝对离差
- `var()`，方差
- `std()`，标准差
- `skew()`，样本值的偏度（三阶矩）
- `kurt()`，样本值的峰度（四阶矩）
- `cumsum()`，样本值的累计和
- `cummin(), cummax()`，	样本值的累计最大值和累计最小值
- `cumprod()`，样本值的累计积
- `diff()`，计算一阶差分（对时间序列很有用）
- `pct_change()`，计算百分数变化

下面给出其中一个方法(`mean()`)的使用语法：
```python
s.mean(axis=None, skipna=None, level=None, numeric_only=None, **kwargs)
df.mean(axis=None, skipna=None, level=None, numeric_only=None, **kwargs)
```
主要参数说明：
- `axis`：行列标记
- `skipna`：是否跳过NaN值
- `level`：对于多层索引有效

In [36]:
# 总收入
df['total_costs'].sum()

34140.0

In [37]:
# 均值
ser.mean()

152.77777777777777

## 查看数据集整体统计信息

使用`describe()`方法，可以一次显示多种统计结果，其使用语法为：
```python
s.describe(percentiles=None, include=None, exclude=None)
df.describe(percentiles=None, include=None, exclude=None)
```
主要参数说明
- `Percentiles`：要显示的百分位数`[.25, .5, .75]`
- `include`：需要显示什么类型的数据
- `exclude`：不需要显示什么类型的数据

In [ ]:
ser.describe()

In [ ]:
df.describe()

In [ ]:
df.describe(include=[np.object])

## 小结

本节介绍查看数据特征以及描述性统计的常用方法：
- `isnull()`，检查数据集是否有空值
- `isna()`，计算数据集是否有非数字值
- `duplicated()`，计算数据集是否有重复
- `unique()`，返回唯一值
- `value_counts()`，频次统计

- `count()`，非 NA 值的数量
- `min(), max()`，计算最小值和最大值
- `argmin(), argmax()`，计算最小值和最大值的索引位置（整数）
- `idxmin(), idxmax()`，计算最小值和最大值的索引值
- `quantile()`，计算样本的百分位数（0到1）
- `sum()`，值得总和
- `mean()`，平均值
- `median()`，中位数
- `mad()`，根据平均值计算平均绝对离差
- `var()`，方差
- `std()`，标准差
- `skew()`，样本值的偏度（三阶矩）
- `kurt()`，样本值的峰度（四阶矩）
- `cumsum()`，样本值的累计和
- `cummin(), cummax()`，	样本值的累计最大值和累计最小值
- `cumprod()`，样本值的累计积
- `diff()`，计算一阶差分（对时间序列很有用）
- `pct_change()`，计算百分数变化
- `describe()`, 查看数据整体统计信息